# Session 1

In the `quickstart_tutorial`, we saw how to control the simulator using a Jupyter Notebook. Now, we will see how to make the agents move and implement basic behaviors.

For this session, we only need one agent in the simulation. To do so, the server can be started in the command line with the options `--n_agents 1` and `n_objects 0`.
<!-- TODO: change this with file system once done !-->
<!-- TODO: set objects to 0 !-->
```bash
python scripts/run_server.py --n_agents 1 --n_objects 0
```

This time, check the `Visible wheels` and `Visible proxs` checkboxes under the **Visualization configurations**.

First of all, we instantiate a `controller` (this will be done at the start of every notebook from now on)

In [ ]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()

This time, only one agent should be present. We can check this with this instruction:

In [ ]:
controller.agents

Next, we start the simulation:

In [ ]:
controller.run(threaded=True)

### Making the agents move

You should now know how to set the attributes of agents. For this time, as there is only one agent, we can create a shortcut for the next cells of code:

In [ ]:
agent = controller.agents[0]

Now, the variable `agent` refers to the first (and only) agent of the simulation. You can simply write `agent` instead of `controller.agents[0]` for the next instructions.

**Q1:** What is the condition for the agent to turn left? to turn right? to move straight forward? to stop?

*This is a cell where you can write text instead of code.* Double click on this text and enter your answer here. Once it is done, press `Shift-Enter`.

**Q2:** How can you make the agent move forward without any rotation? Write the corresponding code in the cell below. 

**Q3:** Now write the code making the agent stop:

**Q4:** And the code to make it move in a large circle:

Let's try more complex choregraphies. A useful function for this is the ability to wait for a given time by using `controller.wait(x)`, where `x` has to be replaced by the time to wait for, in seconds. Here is an example, where the agent goes forward during 3 seconds, then turns right during 0.5 second, then goes forward again during 2 seconds, and finally stops.

In [ ]:
# Move forward, by setting each wheel speed at the same positive value
agent.left_motor = 1.
agent.right_motor = 1.

# Wait for 3 seconds, keeping moving forward at the same speed
controller.wait(3)

# Turn right by setting a positive speed to the left wheel and stoping the right one (null speed)
agent.left_motor = 0.5
agent.right_motor = 0.

# Keep turning for 0.5 second
controller.wait(0.5)

# Move forward again for 2 seconds
agent.left_motor = 1.
agent.right_motor = 1.
controller.wait(2)

# Stop the agent
agent.left_motor = 0.
agent.right_motor = 0.

A few important remarks regarding the code above:
- each line beginning with a `#` symbol corresponds to a comment and will therefore not be executed (it is just to explain verbally what the code does),
- when calling `controller.wait(.)`, the cell that is currently running will simply wait for the given time in seconds before executing the next line of code. this also means that you cannot execute anything else before the wait time is over.

Now, let's repeat the previous choreography 4 times. In the code below, the first line, `for i in range(4):`, means *repeat 4 times the indented code below* (actually it is a bit more complicated than this, but this out of the scope of the current session).  The indented code is the exact same as just before (only the comments have been removed). You can change the number `4` in the first line by any number `x`, to repeat it `x` times instead of 4.

In [ ]:
for i in range(4):
    agent.left_motor = 1.
    agent.right_motor = 1.
    controller.wait(3)
    agent.left_motor = 1.
    agent.right_motor = 0.
    controller.wait(0.5)
    agent.left_motor = 1.
    agent.right_motor = 1.
    controller.wait(2)
    agent.left_motor = 0.
    agent.right_motor = 0.

**Q5:** By executing the code just above, you will observe that the agent actually only stops at the very end of the choreography, but not between each run of the loop. Copy-paste the code of the last cell above in the cell below and add one line of code so that the agent will stop for 1 second at each run of the loop.

**Q6:** Write the code allowing the agent to move roughly in a "8" shape. To do so you'll have to fine tune the waiting times by trial on error. Then make the agent repeat it 6 times.

### Sensors

As shown in the quickstart tutorial, the agents are equipped with proximity sensors, called proximeters that indicate the presence of an entity, if any, in a particular direction. There are two sensor, one on each side of the agent, and their field of view is determined by the `proxs_cos_min` attribute.

Using the commands learned before, move the front of the robot close to a wall in the scene and stop it. Alternatively, you can simply move the agent using drag&drop (see the `web_interface_tutorial.md`).

Whenever a proximeter detects an obstacle, a red point is shown on the agent. Note that each proximeter detects only the closest object within its own field of view (indicate by the red lines). The current activation of the two front proximeters can be accessed with:

In [ ]:
agent.sensors()

This command returns an `array`, i.e. a list of values. Here we see two values: the first one corresponds to the activity of the left proximeter, the second to the activity of the right one. Those values do NOT correspond to distances, but are instead activation values bounded between 0 and 1, where 0 indicates that no obstacle is perceived (i.e. the proximeter doesn't detect any object withing its field of view), and 1 indicates that an object is in contact with the proximeter. In other word, the higher the activation, the closer the object (inversely proportional to the object distance). 

Make sure that you are able to identify which sensor corresponds to which value. 

You can store the values returned by `agent.sensors()` in two distinct variables, which will facilitate using them for further computation. If you execute the following command:

In [ ]:
left, right = agent.sensors()

Then the variables `left` and `right` will contain the activations of the left and right proximeters, respectively. You can check their values using the `print` instruction:

In [ ]:
print(left)

In [ ]:
print(right)

**However**, the values in `left` and `right` will not be updated if the proximeter activations change over time. To experience it, move the robot in a different location in the scene where the proximeter activations should be different, and re-execute the `print` commands above. You will observe that the returned values are the same as before, whereas the proximeter activities should be different. In order to refresh the values in the `left` and `right` variables, you need to re-execute the command `left, right = epuck.prox_activations()`:

In [ ]:
left, right = agent.sensors()
print(left, right)

Now you see that the values have been updated correctly.

**Q7:** To complete this session, write a behavior that connects sensors to motors, so that the robot goes forward at a speed that depends on the summed activity of both proximity sensors. The closer the robot is from an obstacle, the slower it should go (but always going forward, without rotation). Remember that proximeter values have to be refreshed explicitely each time we need to observe them by calling `left, right = epuck.prox_activations()` To do so, use the `for` loop construction we have seen, this time with a large number of iterations. Therefore, the code structure of this behavior will look like:

In [ ]:
# Repeat 1000 times:
for i in range(1000):
    # Read the proximeter values and store them in the left and right variables
    left, right = agent.sensors()
    
    # Write your code below, which has to set the wheel speeds according to the sum left + right
    # Remember to keep your code indented, otherwise it will not be considered as part of the for loop
    # Also remember that the motor values should be between 0 and 1
    

    
    # Keep the line below at the end of the loop. It waits for 100 milliseconds before starting the next iteration (to avoid overloading you computer)
    controller.wait(0.1)

The code above will take a while to be executed (approximately `1000 * 0.1 = 100` seconds). During this time, you can't execute anything else in this notebook. To stop the execution before it terminates by itself, press the "stop-like" button in the top menu bar of this document. 

In the next practical session, we will see more fancy ways of defining reactive behaviours for the robot. And in the next ones, still more and more fancy stuffs.

In [26]:
controller.stop()